## 빅 데이터 분석 및 시각화 개론 프로젝트
# 주제: 대중교통 승하차 데이터를 이용한 우이신설선 수요 예측
## 18조: 장민우, 표성준, 한승규
---
주거구역, 환승구역 등 연구 차원에서 지정한 용어는 발표자료를 참고 부탁드립니다.

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

### 우이신설선 환승역(환승구역) 주변의 모든 버스 정류장
- 서울특별시에서 제공하는 지하철역 근처 버스 정류장 리스트를 활용함.
- 우이신설선 정보가 제공되지 않음에 따라 우이신설선은 별도로 검색 (추후 데이터가 직접 제공될 경우 이를 활용 가능)
- 버스정류장 ID 중 0으로 시작하는 ID의 경우 데이터 정제과정 중 (숫자형 자료형으로 변환되어) 자리수가 변동이 있을 수 있으나, 버스정류장 ID는 기본적으로 다섯자리라는 사전 정보를 감안하여 처리함. 
- 환승 수요가 많으므로 기존 버스 이용 시 반드시 이 역에서 환승하지 않았을 가능성이 있으므로, 실제 환승역을 기준으로 앞/뒤 한 역씩 추가적으로 조사하였음.

In [3]:
subwayToBus = pd.read_csv('subwayToBus.csv')
subwayToBus.head()

,전철역코드,외부코드,전철역명,호선,정류장명,정류장ID,X좌표,Y좌표
0,1958,310,대화,3,예비군훈련장,36262,177627,464279
1,1958,310,대화,3,대화역,36281,177715,464099
2,1958,310,대화,3,대화역,36601,177718,464073
3,1805,150,송내,1,송내역,46039,178169,443269
4,1805,150,송내,1,송내역,46731,178200,443181


In [4]:
transfer_region_list = list()
subway_list = ['성신여대입구','길음','한성대입구','안암','보문','창신','제기동','신설동','동묘앞']
for station in subway_list:
    transfer_region_list.append(subwayToBus.where(subwayToBus['전철역명'] == station).dropna())
transfer_region_list[0]

,전철역코드,외부코드,전철역명,호선,정류장명,정류장ID,X좌표,Y좌표
1088,418.0,418,성신여대입구,4,삼선동주민센터,8176.0,201300.0,454575.0
1089,418.0,418,성신여대입구,4,성신여대입구,8269.0,201435.0,454875.0
1090,418.0,418,성신여대입구,4,성신여대입구,8270.0,201449.0,454884.0
1091,418.0,418,성신여대입구,4,돈암시장입구,8305.0,201461.0,454719.0
1092,418.0,418,성신여대입구,4,돈암시장입구,8306.0,201478.0,454709.0
1093,418.0,418,성신여대입구,4,돈암사거리성신여대입구,8008.0,201576.0,454865.0
1094,418.0,418,성신여대입구,4,돈암사거리성신여대입구,8007.0,201602.0,454913.0


In [5]:
transfer_region = list()
for i in transfer_region_list:
    temp_list = i['정류장ID'].tolist()
    for j in temp_list:
        transfer_region.append(int(j))

### 우이신설선 비환승역(주거구역) 주변 반경 350m 안의 모든 버스 정류장
- 관련 공식 데이터가 준비되지 않아 수동으로 조사함.
- 주거구역이므로 반경 350m(도보 5분) 이내로 기준을 정함.

In [6]:
HomeRegionData = pd.read_csv('HomeRegion.csv')
HomeRegionData.head()

,전철역명,정류장명,정류장ID
0,북한산우이,우이동,9500
1,북한산우이,우이동차고지종점,9294
2,북한산우이,우이동차고지기점,9289
3,북한산우이,우이동도선사입구,9102
4,북한산우이,우이동도선사입구,9101


In [7]:
home_region = list()
temp_list = HomeRegionData['정류장ID'].tolist()
for j in temp_list:
    home_region.append(int(j))

### 환승구역과 주거구역을 잇는 모든 버스 노선들을 검색
- 우이신설선 개통 직전인 2017년 8월, 모든 버스의 정류장별 승하차 인원 데이터를 기반으로, 해당 노선이 지나가는 버스 정류소들을 확인 가능
- 지하철이 운행종료한 후 운행하는 N버스(심야버스) 는 제외하였음

In [8]:
busStationBoarding = pd.read_csv('BUS_STATION_BOARDING_MONTH_201708.csv')
busStationBoarding = busStationBoarding.where(busStationBoarding['사용일자'] == 20170801).dropna()
busStationBoarding.head()

,사용일자,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20170801.0,400,400번(염곡동~시청),102000171.0,03265,국립중앙박물관용산가족공원,130.0,108.0,20170804.0
1,20170801.0,400,400번(염곡동~시청),102000188.0,03282,한국폴리텍1대학,180.0,329.0,20170804.0
2,20170801.0,400,400번(염곡동~시청),102000179.0,03273,서빙고동주민센터,143.0,65.0,20170804.0
3,20170801.0,400,400번(염곡동~시청),102000178.0,03272,서빙고동,37.0,160.0,20170804.0
4,20170801.0,400,400번(염곡동~시청),102000180.0,03274,동빙고동,140.0,152.0,20170804.0


In [9]:
def is_passing_by(region, boarding_df):
    result = boarding_df.where(boarding_df['버스정류장ARS번호'] == '0').dropna()
    for ID in region:
        if ID < 10000:
            IDstr = str(ID).zfill(5)
        else:
            IDstr = str(ID)        
        temp = boarding_df.where(boarding_df['버스정류장ARS번호'] == IDstr).dropna()
        result = result.append(temp,ignore_index=True)

    return result

transfer_region_bus = is_passing_by(transfer_region,busStationBoarding)
transfer_region_bus = transfer_region_bus.reindex(columns=['노선번호']).drop_duplicates()['노선번호'].tolist()

In [10]:
home_region_bus = is_passing_by(home_region, busStationBoarding)
home_region_bus = home_region_bus.reindex(columns=['노선번호']).drop_duplicates()['노선번호'].tolist()

In [11]:
common_line = []
for line in transfer_region_bus:
    if line in home_region_bus:
        if not "N" in line:
            common_line.append(line)
        
print(common_line)

['1162', '2115', '152', '1014', '162', '성북22', '성북20', '143', '104', '151', '171', '109', '1113', '1114', '1164', '7211', '110B', '1213', '153', '110A', '121', '1115', '1128', '101', '144']


### 해당 노선들의 순차적 정류장 목록을 조사
- 해당 정보를 제공하는 웹사이트는 있었으나, 이를 위한 API는 존재하지 않았음.
- 정보가 html이 아닌 javascript로 매번 불러오는 방식이어서 자동화가 매우 힘들었음.
- 따라서 웹사이트를 직접 손으로 크롤링함

In [93]:
buslineStops = pd.read_csv('BuslineStops.csv', encoding="euc_kr")
buslineStopsOriginal = buslineStops.copy()

In [94]:
busline_linenum = buslineStops.columns.tolist()
busline_linelist = list()
for linenum in busline_linenum:
    busline_linelist.append(list())
    current_line = buslineStops[linenum].tolist()
    i = 0
    for station in current_line:
        if type(station) == str:
            busline_linelist[-1].append(station.split('(')[-1].split(')')[0])
            buslineStops[linenum][i] = station.split('(')[-1].split(')')[0]
            i = i + 1

In [95]:
buslineStops.head()

,152,1162,2115,1014,162,성북22,성북20,143,104,171,...,110B,7211,1213,153,110A,121,1115,1128,101,144
0,09211,08367,07418,08258,08161,08846,06591,08161,09194,08113,...,08160,12469,07195,09102,08160,09291,09211,10153,09102,09289
1,09213,08355,07338,08259,08163,08847,08465,08163,09310,08115,...,08161,35160,07194,09103,08161,09137,09213,10154,09103,09102
2,09215,08369,07435,08260,08408,08844,08479,08408,09196,08117,...,08163,12010,07233,09105,08163,09139,09215,10103,10200,09103
3,09218,08370,07433,08262,08165,08842,08494,08165,09111,08119,...,08408,12012,07231,09106,08408,09141,09218,10108,10202,09105
4,09223,08368,07296,08408,08167,08448,08507,08167,09120,08121,...,08165,12111,07229,09107,08165,09156,09223,10110,10204,09106


### 순차 정류장 목록을 이용하여 각 버스별로 어느 구간이 우이신설선과 중복되는지를 검출

In [102]:
busline_common_region = list()
busline_common_df = buslineStops.copy()
for i in range(len(busline_linenum)):
    busline_common_region.append(list())
    current_line = busline_linenum[i]
    j = 0
    for station in busline_linelist[i]:
        if int(station) in home_region:
            busline_common_region[-1].append('home')
            busline_common_df[current_line][j] = 'home'
        elif int(station) in transfer_region:
            busline_common_region[-1].append('transfer')
            busline_common_df[current_line][j] = 'transfer'
        else:
            busline_common_region[-1].append('none')
            busline_common_df[current_line][j] = 'none'
        j = j + 1

In [106]:
busline_common_df

,152,1162,2115,1014,162,성북22,성북20,143,104,171,...,110B,7211,1213,153,110A,121,1115,1128,101,144
0,none,none,none,none,none,none,none,none,none,none,...,none,none,none,home,none,home,none,none,home,home
1,none,none,none,none,home,home,none,home,none,none,...,none,none,none,home,none,home,none,none,home,home
2,none,none,none,home,home,home,none,home,none,home,...,home,none,none,home,home,home,none,none,home,home
3,home,none,none,home,home,home,none,home,home,home,...,home,none,none,home,home,home,home,none,none,home
4,home,none,none,home,home,home,none,home,home,none,...,home,none,none,home,home,home,home,none,none,home
5,home,none,none,home,home,none,none,home,home,none,...,home,none,none,home,home,home,home,none,none,home
6,home,none,none,home,home,none,none,home,home,transfer,...,home,none,none,home,home,home,home,none,none,home
7,home,home,none,home,home,none,none,none,home,none,...,none,none,none,none,home,home,none,none,none,home
8,none,home,none,home,home,home,none,none,home,none,...,none,none,none,none,none,home,none,none,none,home
9,none,home,none,home,home,home,none,transfer,home,none,...,none,none,none,none,transfer,home,none,none,none,home
